# 교통 데이터 수집 및 전처리

In [ ]:
import pandas as pd
ac = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권-집객시설).csv', encoding='cp949')
ac

In [ ]:
mask1= ac[(ac['상권_구분_코드'].isin(['A','D'])) & (ac['기준_년_코드'] == 2021) & (ac['기준_분기_코드'] == 4)]
mask1

## 상권코드 & 행정동코드 합치고 그룹바이

In [ ]:
ad = pd.read_csv('상권코드_행정동코드.csv')
gs = pd.merge(mask1, ad, how='right', on='상권_코드')
gs

In [ ]:
gs = gs.drop(['Unnamed: 0'], axis = 1) 

In [ ]:
gs = gs.groupby(['행정동_코드']).agg({"철도_역_수":"sum","버스_터미널_수":"sum",
                                 "지하철_역_수" : "sum","버스_정거장_수" : "sum",})

In [ ]:
gs = gs.astype('int')  ##소수점으로 표시돼서 정수로 변환
gs

In [ ]:
gs.to_csv('교통데이터_최종.csv', encoding='cp949')

# 데이터셋 병합

In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_csv('점밀_프수_여비_외인_문수2.csv')
mc = pd.read_csv('추정매출_임대료.csv', encoding='ANSI')

In [ ]:
df2.drop('Unnamed: 0', axis=1, inplace=True)
mc.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
print(df2.info())
print(mc.info())

In [ ]:
df2.rename(columns = {'행정동코드' : '행정동_코드'}, inplace = True)

In [ ]:
df = pd.merge(mc, df2, how='inner', on='행정동_코드')

In [ ]:
df.info()

### 컬럼 추출

In [ ]:
df.columns

Index(['행정동_코드', '상권_구분_코드', '상권_코드', '분기당_매출_금액', '분기당_매출_건수', '건당_매출액_평균',
       '환산임대료(원, 21년도)', '증감율(%, 19-21년도)', '임대료/매출', '매출/임대료', '행정동명', '점포수',
       '면적', '단위면적당 상권 수(k㎡)', '프랜차이즈_점포_수', '개인사업자_수', '프랜차이즈 비율',
       '1월_2030여성비율', '2월_2030여성비율', '3월_2030여성비율', '4월_2030여성비율',
       '5월_2030여성비율', '6월_2030여성비율', '7월_2030여성비율', '8월_2030여성비율',
       '9월_2030여성비율', '10월_2030여성비율', '11월_2030여성비율', '12월_2030여성비율',
       '2021년_2030여성_비율평균', '1월_외국인_총생활인구수', '2월_외국인_총생활인구수', '3월_외국인_총생활인구수',
       '4월_외국인_총생활인구수', '5월_외국인_총생활인구수', '6월_외국인_총생활인구수', '7월_외국인_총생활인구수',
       '8월_외국인_총생활인구수', '9월_외국인_총생활인구수', '10월_외국인_총생활인구수', '11월_외국인_총생활인구수',
       '12월_외국인_총생활인구수', '2021년_외국인_평균', '문화시설수'],
      dtype='object')

In [ ]:
df = df[['행정동_코드', '행정동명', '상권_구분_코드', '상권_코드', '분기당_매출_금액', '분기당_매출_건수', '건당_매출액_평균',
       '환산임대료(원, 21년도)', '증감율(%, 19-21년도)', '임대료/매출', '매출/임대료', '점포수',
       '면적', '단위면적당 상권 수(k㎡)', '프랜차이즈_점포_수', '개인사업자_수', '프랜차이즈 비율',
       '2021년_2030여성_비율평균', '2021년_외국인_평균', '문화시설수']]

In [ ]:
df.info()

#### csv파일로 저장

In [ ]:
df.to_csv('./최종본_edit.csv')

#위의 병합된 데이터에 교통데이터 변수 병합 및 전처리(결측치)

In [ ]:
#전처리 및 1차 병합을 완료한 데이터 불러오기 (행정동별 점포밀도+프렌차이즈 수)
df=pd.read_csv('면적_프렌차이즈.csv')
df

In [ ]:
# 불필요한 열 삭제
del df['Unnamed: 0']

In [ ]:
#전처리가 완료된 행정동별 교통데이터 불러오기
df_traffic=pd.read_csv('교통데이터_최종.csv', encoding='cp949')
df_traffic
#점포밀도+프렌차이즈 수 데이터와 비교했을때 없 행(행정동)이 있음을 확인

In [ ]:
#없는 행(행정동)을 찾기 위해 우선 행정동명 열을 추가
dong_name = []
for i in range(len(df_traffic)):
    name = df[df['행정동코드'] == df_traffic.iloc[i,0]]['행정동명'].iloc[0]
    dong_name.append(name)
dong_name
#single positional indexer is out-of-bounds 범위를 벗어낫다는 에러가 뜬다
#아마 행이 하나인 데이터프레임에서 iloc함수를 반복문을 돌려서 그런거같다.
#따라서 결과값이 여러줄이 뜨도록 서울소상공인상권데이터를 가져와서 돌려보려고 한다.

In [ ]:
#서울소상공인상권데이터를 가져오기
df_=pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202112.csv')

In [ ]:
#행정동명을 찾기 위해 전처리된 데이터프레임이 아닌 서울소상공인상권데이터로 분석.
# 100을 곱해서 넣어준 이유는 행정동코드 10자리를 맞추기 위해서이다.
dong_name = []
for i in range(len(df_traffic)):
    name = df_[df_['행정동코드'] == df_traffic.iloc[i,0]*100]['행정동명'].iloc[0]
    dong_name.append(name)
dong_name
#값이 들어옴을 확인.

In [ ]:
df_traffic['행정동명'] = dong_name
df_traffic

In [ ]:
#일치하게 들어왔는지 확인
df_[df_['행정동코드'] == 1111051500]['행정동명'].iloc[0] == df_traffic[df_traffic['행정동_코드']== 11110515]['행정동명'].iloc[0]

In [ ]:
df_traffic[df_traffic['행정동명'] == '신사동']
#이 데이터에도 신사동이 구분되지 않았기에 구분이 필요하다.

In [ ]:
df[df['행정동코드'] == 11620685]
#11620685 행정동 코드가 관악구임을 확인

In [ ]:
df_traffic.loc[320, '행정동명'] = '신사동(관악구)'
df_traffic.loc[343, '행정동명'] = '신사동(강남구)'
#데이터 구분되게 수정

In [ ]:
df_traffic[df_traffic['행정동명'].str.contains('신사동')]
#구분됨을 확인

In [ ]:
#행정동_코드 컬럼명 변경
df_traffic = df_traffic.rename(columns={'행정동_코드':'행정동코드'})

In [ ]:
#'행정동코드' 와 '행정동명'을 기준으로 밀도_프렌차이즈 데이터와 교통 데이터 테스트 병합
df_test1 = pd.merge(df,df_traffic, how='outer',on=['행정동코드','행정동명'])#merge함수로 병합

In [ ]:
df_test1.info()
#행정동코드와 행정동명 길이가 늘어난 것을 보니 행정동 코드가 서로다른 데이터가 있음을 확인.

In [ ]:
df_test1[df_test1['점포수'].isnull()]
#결측치확인
#늘어낭 행의 길이만큼 행정동코그가 불일치하는 데이터 확인
#실제 소상공인상권정보 데이터에서, 같은 행정동이지만 행정동코드가 조금씩 다른 부분이 있다.

In [ ]:
#행정동코드를 통해 행정동명 찾기 
# df_(서울소상공인상권데이터)에서는 10자리 코드를 사용하기에 100을 곱해준다.
print(df_[df_['행정동코드'] == 1130559000]['행정동명'].iloc[0]) #번1동 확인

#병합할때 쓰였던 전처리가 완료된 df_new_mildo데이터 프레임에서의 번1동 행정동코드 확인
print(df[df['행정동명'] == '번1동']['행정동코드'].iloc[0])

#출력을 보면 같은 번1동이지만 행정동코드 끝자리가 다름을 확인 이를 수정처리
# 병합할때 쓰였던 전처리가 완료된 df_traffic(교통데이터)에서 수정.

print(df_traffic[df_traffic['행정동코드'] == 11305590].index) #인덱스번호 출력
df_traffic.loc[132, '행정동코드'] = 11305595 #행정동코드 수정
print(df_traffic.loc[132, '행정동코드'] == 11305595) # 바뀌었음을 확인

#같은 방법으로 다른 행정동 코드 수정.

In [ ]:
print(df_[df_['행정동코드'] == 1130560000]['행정동명'].iloc[0]) #번2동 확인
print(df[df['행정동명'] == '번2동']['행정동코드'].iloc[0])
print(df_traffic[df_traffic['행정동코드'] == 11305600].index) #인덱스번호 출력
df_traffic.loc[133, '행정동코드'] = 11305603 #수정
print(df_traffic.loc[133, '행정동코드'] == 11305603) # 바뀌었음을 확인

In [ ]:
print(df_[df_['행정동코드'] == 1130560600]['행정동명'].iloc[0]) #번3동 확인
print(df[df['행정동명'] == '번3동']['행정동코드'].iloc[0])
print(df_traffic[df_traffic['행정동코드'] == 11305606].index) #인덱스번호 출력
df_traffic.loc[134, '행정동코드'] = 11305608 #수정
print(df_traffic.loc[134, '행정동코드'] == 11305608) # 바뀌었음을 확인

In [ ]:
print(df_[df_['행정동코드'] == 1130561000]['행정동명'].iloc[0]) #수유1동 확인
print(df[df['행정동명'] == '수유1동']['행정동코드'].iloc[0])
print(df_traffic[df_traffic['행정동코드'] == 11305610].index) #인덱스번호 출력
df_traffic.loc[135, '행정동코드'] = 11305615 #수정
print(df_traffic.loc[135, '행정동코드'] == 11305615) # 바뀌었음을 확인

In [ ]:
print(df_[df_['행정동코드'] == 1130562000]['행정동명'].iloc[0]) #수유2동 확인
print(df[df['행정동명'] == '수유2동']['행정동코드'].iloc[0])
print(df_traffic[df_traffic['행정동코드'] == 11305620].index) #인덱스번호 출력
df_traffic.loc[136, '행정동코드'] = 11305625 #수정
print(df_traffic.loc[136, '행정동코드'] == 11305625) # 바뀌었음을 확인

In [ ]:
print(df_[df_['행정동코드'] == 1130563000]['행정동명'].iloc[0]) #수유3동 확인
print(df[df['행정동명'] == '수유3동']['행정동코드'].iloc[0])
print(df_traffic[df_traffic['행정동코드'] == 11305630].index) #인덱스번호 출력
df_traffic.loc[137, '행정동코드'] = 11305635 #수정
print(df_traffic.loc[137, '행정동코드'] == 11305635) # 바뀌었음을 확인

In [ ]:
# 행정동코드를 수정하였으니 다시 병합을 진행.
df_test2 = pd.merge(df,df_traffic, how='outer',on=['행정동코드','행정동명'])
#merge함수로 병합

In [ ]:
df_test2.info()
#두 데이터프레임의 행정동코드 데이터를 일치시켜 줌으로서 결측치는 한쪽에서만 생김.

In [ ]:
df_test2[df_test2['행정동명'].isnull()]
#왼쪽 데이터프레임 결측치는 없다.

In [ ]:
df_test2[df_test2['철도_역_수'].isnull()]
#오른쪽 데이터 프레임 결측치 확인

In [ ]:
len(df_test2[df_test2['철도_역_수'].isnull()])

In [ ]:
#교통데이터 수정( 버스정거장과 지하철역이 있지만 0으로 표기된 데이터들 수정.)
#데이터 수집이 어려워서 네이버 지도를 통해 확인하였다.
df_traffic.loc[183, '지하철_역_수'] = 3 #수색동
df_traffic.loc[183, '버스_정거장_수'] = 20 #수색동

df_traffic.loc[230, '지하철_역_수'] = 1 #신정7동
df_traffic.loc[230, '버스_정거장_수'] = 19 #신정7동

df_traffic.loc[363, '지하철_역_수'] = 1 #풍납1동
df_traffic.loc[363, '버스_정거장_수'] = 6 #풍납1동

df_traffic.loc[382, '지하철_역_수'] = 1 #위례동
df_traffic.loc[382, '버스_정거장_수'] = 27 #위례동

In [ ]:
#교통데이터 결측치 추가.
#데이터 수집이 어려워서 네이버 지도를 통해 확인하였다.
#편의성을 위해 input을 활용해서 바로바로 데이터프레임에 추가해 주었다.
dong_code = input('행정동코드: ')
subway = input('지하철역수: ')
buss = input('버스정류장수: ')
dong = input('행정동명: ')

traffic_data = {   
                '행정동코드': dong_code,
                '철도_역_수': 0,
                '버스_터미널_수': 0,
                '지하철_역_수': subway,
                '버스_정거장_수': buss,
                '행정동명': dong
                }
df_traffic = df_traffic.append(traffic_data, ignore_index=True)
df_traffic

In [ ]:
#자료형변환
df_traffic = df_traffic.astype({'행정동코드':'int'})
df_traffic

In [ ]:
#df_traffic.to_csv('교통데이터찐최종.csv', encoding="utf-8-sig")

#최종병합

In [ ]:
#파일불러오기
df_final=pd.read_csv('최종본_edit.csv', encoding='cp949')
df_final

In [ ]:
del df_final['Unnamed: 0']
df_final

In [ ]:
df_traffic=pd.read_csv('교통데이터찐최종.csv')
df_traffic

In [ ]:
del df_traffic['Unnamed: 0']

In [ ]:
df_test7 = pd.merge(df_final,df_traffic, how='inner')#merge함수로 병합 최종+교통
df_test7

In [ ]:
#철도역/버스터미널열/행정동코드 삭제-> 행정동_코드 열 존재
del df_test7['행정동코드']
del df_test7['철도_역_수']
del df_test7['버스_터미널_수']

In [ ]:
#최종+교통 csv로
#df_test7.to_csv('최종_교통추가.csv', encoding="utf-8-sig")

# 점포수 변수 추가

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('최종_교통추가.csv')
df2 = pd.read_csv('점포수.csv')

In [ ]:
df1.drop(['Unnamed: 0', '개업률_평균', '폐업률_평균'], axis=1, inplace=True)
df2.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df = pd.merge(df1, df2, how='inner', on='상권_코드')
df = df.fillna(0)

In [ ]:
df.info()

#### csv파일로 저장

In [ ]:
#df.to_csv('./final.csv')

# 상권코드명 추가

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('final.csv')
df2 = pd.read_csv('상권코드명.csv')

In [ ]:
df1.drop('Unnamed: 0', axis=1, inplace=True)
df2.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   행정동_코드             1260 non-null   int64  
 1   행정동명               1260 non-null   object 
 2   상권_구분_코드           1260 non-null   object 
 3   상권_코드              1260 non-null   int64  
 4   분기당_매출_금액          1260 non-null   int64  
 5   분기당_매출_건수          1260 non-null   int64  
 6   건당_매출액_평균          1260 non-null   int64  
 7   환산임대료(원, 21년도)     1260 non-null   int64  
 8   증감율(%, 19-21년도)    1260 non-null   float64
 9   임대료/매출             1260 non-null   float64
 10  매출/임대료             1260 non-null   float64
 11  점포수                1260 non-null   int64  
 12  면적                 1260 non-null   float64
 13  단위면적당 상권 수(k㎡)     1260 non-null   float64
 14  프랜차이즈_점포_수         1260 non-null   int64  
 15  개인사업자_수            1260 non-null   int64  
 16  프랜차이즈 비율           1260 

In [ ]:
df = pd.merge(df1, df2, how='inner', on='상권_코드')

In [ ]:
df.columns

Index(['행정동_코드', '행정동명', '상권_구분_코드', '상권_코드', '분기당_매출_금액', '분기당_매출_건수',
       '건당_매출액_평균', '환산임대료(원, 21년도)', '증감율(%, 19-21년도)', '임대료/매출', '매출/임대료',
       '점포수', '면적', '단위면적당 상권 수(k㎡)', '프랜차이즈_점포_수', '개인사업자_수', '프랜차이즈 비율',
       '2021년_2030여성_비율평균', '2021년_외국인_평균', '문화시설수', '지하철_역_수', '버스_정거장_수',
       '유사_업종_점포_수', '개인사업자_점포수', '개인사업자_비율', '상권_코드_명'],
      dtype='object')

In [ ]:
df = df[['행정동_코드', '행정동명', '상권_구분_코드', '상권_코드', '상권_코드_명', '분기당_매출_금액', '분기당_매출_건수',
       '건당_매출액_평균', '환산임대료(원, 21년도)', '증감율(%, 19-21년도)', '임대료/매출', '매출/임대료',
       '점포수', '면적', '단위면적당 상권 수(k㎡)', '프랜차이즈_점포_수',
       '개인사업자_수', '프랜차이즈 비율', '2021년_2030여성_비율평균', '2021년_외국인_평균', '문화시설수',
       '지하철_역_수', '버스_정거장_수', '유사_업종_점포_수', '개인사업자_점포수', '개인사업자_비율']]

In [ ]:
df.info()

# 컬럼명 변경

In [ ]:
df.rename(columns = {"유사_업종_점포_수": "전체_카페_점포_수", "개인사업자_점포수": "개인카페_점포수", "개인사업자_비율":"개인카페_비율", "단위면적당 상권 수(k㎡)":"단위면적당 점포 수(k㎡)"}, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260 entries, 0 to 1259
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   행정동_코드             1260 non-null   int64  
 1   행정동명               1260 non-null   object 
 2   상권_구분_코드           1260 non-null   object 
 3   상권_코드              1260 non-null   int64  
 4   상권_코드_명            1260 non-null   object 
 5   분기당_매출_금액          1260 non-null   int64  
 6   분기당_매출_건수          1260 non-null   int64  
 7   건당_매출액_평균          1260 non-null   int64  
 8   환산임대료(원, 21년도)     1260 non-null   int64  
 9   증감율(%, 19-21년도)    1260 non-null   float64
 10  임대료/매출             1260 non-null   float64
 11  매출/임대료             1260 non-null   float64
 12  점포수                1260 non-null   int64  
 13  면적                 1260 non-null   float64
 14  단위면적당 점포 수(k㎡)     1260 non-null   float64
 15  프랜차이즈_점포_수         1260 non-null   int64  
 16  개인사업자_수            1260 

In [ ]:
#df.to_csv('./dataset.csv')

# 활성화 여부 (종속변수) 추가

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('dataset.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
y = pd.read_csv('sns_test_hm.csv')
y.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
y

,활성화_여부
0,2
1,2
2,2
3,2
4,2
...,...
1255,1
1256,2
1257,2
1258,2


In [ ]:
df = pd.concat([df, y], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260 entries, 0 to 1259
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   행정동_코드             1260 non-null   int64  
 1   행정동명               1260 non-null   object 
 2   상권_구분_코드           1260 non-null   object 
 3   상권_코드              1260 non-null   int64  
 4   상권_코드_명            1260 non-null   object 
 5   분기당_매출_금액          1260 non-null   int64  
 6   분기당_매출_건수          1260 non-null   int64  
 7   건당_매출액_평균          1260 non-null   int64  
 8   환산임대료(원, 21년도)     1260 non-null   int64  
 9   증감율(%, 19-21년도)    1260 non-null   float64
 10  임대료/매출             1260 non-null   float64
 11  매출/임대료             1260 non-null   float64
 12  점포수                1260 non-null   int64  
 13  면적                 1260 non-null   float64
 14  단위면적당 점포 수(k㎡)     1260 non-null   float64
 15  프랜차이즈_점포_수         1260 non-null   int64  
 16  개인사업자_수            1260 

In [ ]:
df.to_csv('./dataset_final.csv')